In [1]:
%load_ext autoreload
%autoreload 2

# Tinygrid ERCOT API Demo

This notebook demonstrates how to use tinygrid to access ERCOT grid data with a clean, unified API.

**Features:**
- Type-safe enums for markets and location types
- Convenient date handling with keywords like "today" and "yesterday"
- Unified methods that route to the correct endpoints automatically
- Built-in location filtering

## Setup

First, set your ERCOT API credentials as environment variables:

```bash
export ERCOT_USERNAME="your-email@example.com"
export ERCOT_PASSWORD="your-password"
export ERCOT_SUBSCRIPTION_KEY="your-subscription-key"
```

Or create a `.env` file in the examples directory.

In [2]:
import os

import pandas as pd
from dotenv import load_dotenv

from tinygrid import ERCOT, ERCOTAuth, ERCOTAuthConfig, LocationType, Market

# Load environment variables from .env file
load_dotenv()
pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 200)

## Create Authenticated Client

In [3]:
# Create authenticated client
auth = ERCOTAuth(
    ERCOTAuthConfig(
        username=os.environ["ERCOT_USERNAME"],
        password=os.environ["ERCOT_PASSWORD"],
        subscription_key=os.environ["ERCOT_SUBSCRIPTION_KEY"],
    )
)

ercot = ERCOT(auth=auth)
print("Client created successfully!")

Client created successfully!


## Market and Location Types

Tinygrid uses enums for type safety and IDE autocomplete.

In [4]:
print("Available Markets:")
for m in Market:
    print(f"  Market.{m.name}")

print("\nAvailable Location Types:")
for lt in LocationType:
    print(f"  LocationType.{lt.name}")

Available Markets:
  Market.REAL_TIME_SCED
  Market.REAL_TIME_15_MIN
  Market.DAY_AHEAD_HOURLY

Available Location Types:
  LocationType.LOAD_ZONE
  LocationType.TRADING_HUB
  LocationType.RESOURCE_NODE
  LocationType.ELECTRICAL_BUS


## Settlement Point Prices (SPP)

Get real-time or day-ahead settlement point prices with optional filtering.

In [5]:
# Real-time 15-minute SPP
df = ercot.get_spp(
    start="2024-12-12",
    market=Market.REAL_TIME_15_MIN,
)

print(f"Real-Time SPP: {len(df):,} records")
df.head()

Real-Time SPP: 91,390 records


,Time,End Time,Location,Price,Market,Location Type
0,2024-12-12 23:30:00-06:00,2024-12-12 23:45:00-06:00,7RNCHSLR_ALL,18.81,REAL_TIME_15_MIN,RN
1,2024-12-12 23:30:00-06:00,2024-12-12 23:45:00-06:00,ADL_RN,20.58,REAL_TIME_15_MIN,RN
2,2024-12-12 23:30:00-06:00,2024-12-12 23:45:00-06:00,AEEC,8.81,REAL_TIME_15_MIN,RN
3,2024-12-12 23:30:00-06:00,2024-12-12 23:45:00-06:00,AE_RN,18.83,REAL_TIME_15_MIN,RN
4,2024-12-12 23:30:00-06:00,2024-12-12 23:45:00-06:00,AGUAYO_UNIT1,8.88,REAL_TIME_15_MIN,RN


In [6]:
# Filter to Load Zones only
df = ercot.get_spp(
    start="yesterday",
    market=Market.REAL_TIME_15_MIN,
    location_type=LocationType.LOAD_ZONE,
)

print(f"Load Zone SPP: {len(df):,} records")
df.head()

Load Zone SPP: 1,520 records


,Time,End Time,Location,Price,Market,Location Type
0,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_AEN,8.10,REAL_TIME_15_MIN,LZ
1,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_AEN,8.10,REAL_TIME_15_MIN,LZEW
2,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_CPS,7.73,REAL_TIME_15_MIN,LZEW
3,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_CPS,7.73,REAL_TIME_15_MIN,LZ
4,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_HOUSTON,9.89,REAL_TIME_15_MIN,LZEW


In [7]:
# Filter to specific locations
df = ercot.get_spp(
    start="yesterday",
    market=Market.REAL_TIME_15_MIN,
    locations=["LZ_HOUSTON", "LZ_NORTH", "HB_HOUSTON"],
)

print(f"Specific locations: {len(df):,} records")
df

Specific locations: 475 records


,Time,End Time,Location,Price,Market,Location Type
0,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,HB_HOUSTON,9.96,REAL_TIME_15_MIN,HU
1,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_HOUSTON,9.89,REAL_TIME_15_MIN,LZEW
2,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_HOUSTON,9.89,REAL_TIME_15_MIN,LZ
3,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_NORTH,9.87,REAL_TIME_15_MIN,LZ
4,2025-12-27 23:30:00-06:00,2025-12-27 23:45:00-06:00,LZ_NORTH,9.88,REAL_TIME_15_MIN,LZEW
...,...,...,...,...,...,...
470,2025-12-27 00:00:00-06:00,2025-12-27 00:15:00-06:00,HB_HOUSTON,13.94,REAL_TIME_15_MIN,HU
471,2025-12-27 00:00:00-06:00,2025-12-27 00:15:00-06:00,LZ_HOUSTON,13.93,REAL_TIME_15_MIN,LZ
472,2025-12-27 00:00:00-06:00,2025-12-27 00:15:00-06:00,LZ_HOUSTON,13.93,REAL_TIME_15_MIN,LZEW
473,2025-12-27 00:00:00-06:00,2025-12-27 00:15:00-06:00,LZ_NORTH,13.97,REAL_TIME_15_MIN,LZEW


In [8]:
# Day-Ahead SPP
df = ercot.get_spp(
    start="2025-12-28",
    market=Market.DAY_AHEAD_HOURLY,
    location_type=[LocationType.LOAD_ZONE, LocationType.TRADING_HUB],
)

print(f"Day-Ahead SPP: {len(df):,} records")
df

Day-Ahead SPP: 360 records


,Time,End Time,Location,Price,Market
0,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,HB_BUSAVG,12.61,DAY_AHEAD_HOURLY
1,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,HB_HOUSTON,12.65,DAY_AHEAD_HOURLY
2,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,HB_HUBAVG,10.58,DAY_AHEAD_HOURLY
3,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,HB_NORTH,15.98,DAY_AHEAD_HOURLY
4,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,HB_PAN,-2.69,DAY_AHEAD_HOURLY
...,...,...,...,...,...
355,2025-12-28 23:00:00-06:00,2025-12-29 00:00:00-06:00,LZ_LCRA,13.70,DAY_AHEAD_HOURLY
356,2025-12-28 23:00:00-06:00,2025-12-29 00:00:00-06:00,LZ_NORTH,15.12,DAY_AHEAD_HOURLY
357,2025-12-28 23:00:00-06:00,2025-12-29 00:00:00-06:00,LZ_RAYBN,18.46,DAY_AHEAD_HOURLY
358,2025-12-28 23:00:00-06:00,2025-12-29 00:00:00-06:00,LZ_SOUTH,14.25,DAY_AHEAD_HOURLY


## Locational Marginal Prices (LMP)

Get LMP data by settlement point or electrical bus.

In [9]:
# Real-time LMP by settlement point
df = ercot.get_lmp(
    start="today",
    market=Market.REAL_TIME_SCED,
)

print(f"Real-Time LMP: {len(df):,} records")
df.head()

Real-Time LMP: 50,000 records


,Location,Price,Market,SCED Time Stamp,Repeat Hour Flag
0,BASTEN_CCU,3.55,REAL_TIME_SCED,2025-12-28T12:00:19,False
1,BATCAVE_RN,2.67,REAL_TIME_SCED,2025-12-28T12:00:19,False
2,BAYC_BESS_RN,3.69,REAL_TIME_SCED,2025-12-28T12:00:19,False
3,BBREEZE_1_2,-2.74,REAL_TIME_SCED,2025-12-28T12:00:19,False
4,BCATWD_WD_1,-8.86,REAL_TIME_SCED,2025-12-28T12:00:19,False


In [10]:
# LMP by electrical bus (more granular)
df = ercot.get_lmp(
    start="today",
    market=Market.REAL_TIME_SCED,
    location_type=LocationType.ELECTRICAL_BUS,
)

print(f"Electrical Bus LMP: {len(df):,} records")
df.head()

Electrical Bus LMP: 220,000 records


,Price,Market,SCED Time Stamp,Repeat Hour Flag,Electrical Bus
0,2.11,REAL_TIME_SCED,2025-12-28T12:05:16,False,TPR345BUS1
1,2.11,REAL_TIME_SCED,2025-12-28T12:05:16,False,TPR345BUS2
2,2.14,REAL_TIME_SCED,2025-12-28T12:05:16,False,TPR5TR1
3,2.14,REAL_TIME_SCED,2025-12-28T12:05:16,False,TPRMOBILE
4,4.43,REAL_TIME_SCED,2025-12-28T12:05:16,False,TR162


In [13]:
# Day-Ahead LMP
df = ercot.get_lmp(
    start="2025-12-20",
    market=Market.DAY_AHEAD_HOURLY,
)

print(f"Day-Ahead LMP: {len(df):,} records")
df.head()

Day-Ahead LMP: 0 records


,DeliveryDate,HourEnding,BusName,LMP,DSTFlag


## Ancillary Services

In [14]:
# Ancillary Service Prices (MCPC)
df = ercot.get_as_prices(start="yesterday")

print(f"AS Prices: {len(df):,} records")
df.head()

AS Prices: 120 records


,Time,End Time,Ancillary Type,MCPC
0,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,NSPIN,3.00
1,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,RRS,0.41
2,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,REGDN,0.69
3,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,REGUP,0.41
4,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,ECRS,0.50


In [28]:
# Ancillary Service Plan (requirements by hour)
df = ercot.get_as_plan(start="yesterday")

print(f"AS Plan: {len(df):,} records")
df.head()

AS Plan: 840 records


,Time,End Time,Posted,Ancillary Type,Quantity
0,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,2025-12-27T05:00:00,ECRS,864
1,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,2025-12-27T05:00:00,NSPIN,2278
2,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,2025-12-27T05:00:00,REGDN,315
3,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,2025-12-27T05:00:00,REGUP,369
4,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,2025-12-27T05:00:00,RRS,2982


## Shadow Prices

In [16]:
# SCED Shadow Prices (real-time)
df = ercot.get_shadow_prices(
    start="yesterday",
    market=Market.REAL_TIME_SCED,
)

print(f"SCED Shadow Prices: {len(df):,} records")
df.head()

SCED Shadow Prices: 2,011 records


,SCEDTimeStamp,Constraint ID,Constraint Name,Contingency Name,Shadow Price,Max Shadow Price,Limit,Value,Violated MW,From Station,To Station,From Station kV,To Station kV,CCT Status
0,12/27/2025 22:55:15,6,PNHNDL,BASE CASE,0.00000,5251,2369.3,2217.1,-152.2,NaN,NaN,0,0,NONCOMP
1,12/27/2025 22:55:15,9,BLESSI_LOLITA1_1,DSTEXP12,0.00000,3500,209.6,178.7,-31.0,LOLITA,BLESSING,138,138,COMP
2,12/27/2025 22:55:15,7,STPELM27_1,DSTEXP12,0.00000,4500,613.2,542.4,-70.8,ELMCREEK,STP,345,345,COMP
3,12/27/2025 22:55:15,2,587__A,MRNKDHM5,511.61374,3500,210.4,210.4,0.0,ARGYL,LWSVH,138,138,NONCOMP
4,12/27/2025 22:55:15,8,940__A,SPEBTRU8,0.00000,3500,250.7,217.3,-33.5,ENWSW,TMPTN,138,138,NONCOMP


In [17]:
# Day-Ahead Shadow Prices
df = ercot.get_shadow_prices(
    start="yesterday",
    market=Market.DAY_AHEAD_HOURLY,
)

print(f"DAM Shadow Prices: {len(df):,} records")
df.head()

DAM Shadow Prices: 730 records


,Time,End Time,Constraint Id,Constraint Name,Contingency Name,Constraint Limit,Constraint Value,Violation Amount,Shadow Price,From Station,To Station,From Station kV,To Station kV,Delivery Time
0,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,1,PNHNDL,BASE CASE,2480,2480,0,9.970,,,0.0,0.0,2025-12-27T01:00:00
1,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,2,DEC_G1NX,BASE CASE,70,70,0,0.010,DEC,DEC,13.8,1.0,2025-12-27T01:00:00
2,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,3,587__A,MRNKDHM5,220,220,0,218.951,ARGYL,LWSVH,138.0,138.0,2025-12-27T01:00:00
3,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,4,HARGRO_TWINBU1_1,DBAKCED5,151,151,0,14.268,TWINBU,HARGROVE,138.0,138.0,2025-12-27T01:00:00
4,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,5,6965__A,DBAKCED5,1331,1331,0,35.197,LNGSW,PRLSW,345.0,345.0,2025-12-27T01:00:00


## System Load

In [27]:
# Load by weather zone
df = ercot.get_load(start="yesterday", by="weather_zone")

print(f"System Load: {len(df):,} records")
df

System Load: 24 records


,Operating Day,Coast,East,Far West,North,NorthC,Southern,SouthC,West,Total
0,2025-12-27,11909.01,1489.90,7621.24,1660.80,11756.22,3711.73,6938.10,1115.55,46202.56
1,2025-12-27,11489.27,1410.69,7710.51,1645.75,11217.10,3589.45,6646.79,1014.51,44724.07
2,2025-12-27,11185.85,1438.05,7719.28,1640.99,10756.89,3540.12,6463.25,1007.82,43752.26
3,2025-12-27,11005.67,1383.01,7610.70,1601.40,10588.78,3560.05,6361.05,1024.33,43134.98
4,2025-12-27,10928.55,1369.79,7615.60,1527.04,10440.07,3484.01,6333.61,1016.33,42715.00
5,2025-12-27,10969.61,1382.33,7663.91,1504.64,10544.05,3501.95,6336.24,1013.21,42915.94
6,2025-12-27,11108.92,1411.45,7661.25,1526.86,10729.00,3555.23,6485.66,1049.17,43527.54
7,2025-12-27,11205.97,1445.58,7640.94,1539.64,11054.72,3564.17,6637.37,1069.53,44157.92
8,2025-12-27,11562.15,1486.75,7532.29,1583.00,11649.28,3654.30,6886.24,1179.16,45533.16
9,2025-12-27,12107.21,1512.00,7471.27,1647.92,12321.99,3948.85,7195.08,1280.05,47484.37


## Wind & Solar Forecasts

In [24]:
# Wind forecast
df = ercot.get_wind_forecast(start="2025-12-28")

print(f"Wind Forecast: {len(df):,} records")
df.tail(n=100)

Wind Forecast: 288 records


,Time,End Time,Posted,Generation System Wide,COP HSL System Wide,STWPF System Wide,WGRPP System Wide,Generation Load Zone South Houston,COP HSL Load Zone South Houston,STWPF Load Zone South Houston,WGRPP Load Zone South Houston,Generation Load Zone West,COP HSL Load Zone West,STWPF Load Zone West,WGRPP Load Zone West,Generation Load Zone North,COP HSL Load Zone North,STWPF Load Zone North,WGRPP Load Zone North,HSL System Wide
188,2025-12-28 20:00:00-06:00,2025-12-28 21:00:00-06:00,2025-12-28T04:55:33,NaN,24687.8,25246.2,23500.5,NaN,3172.1,3192.9,2738.6,NaN,18617.7,19155.3,18015.7,NaN,2898.0,2898.0,2746.2,NaN
189,2025-12-28 21:00:00-06:00,2025-12-28 22:00:00-06:00,2025-12-28T04:55:33,NaN,22505.5,23066.9,23066.9,NaN,2608.9,2619.4,2619.4,NaN,17123.3,17674.2,17674.2,NaN,2773.3,2773.3,2773.3,NaN
190,2025-12-28 22:00:00-06:00,2025-12-28 23:00:00-06:00,2025-12-28T04:55:33,NaN,21594.4,22144.9,22144.9,NaN,2380.0,2387.7,2387.7,NaN,16453.4,16996.2,16996.2,NaN,2761.0,2761.0,2761.0,NaN
191,2025-12-28 23:00:00-06:00,2025-12-29 00:00:00-06:00,2025-12-28T04:55:33,NaN,20320.5,20867.5,20867.5,NaN,2102.2,2107.9,2107.9,NaN,15512.4,16053.7,16053.7,NaN,2705.9,2705.9,2705.9,NaN
192,2025-12-28 00:00:00-06:00,2025-12-28 01:00:00-06:00,2025-12-28T03:55:34,25025.58,27712.8,28230.0,27154.4,6415.52,6417.5,6662.4,6381.7,15841.16,18403.1,18666.2,17964.9,2768.9,2892.2,2901.4,2807.8,28274.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,2025-12-28 19:00:00-06:00,2025-12-28 20:00:00-06:00,2025-12-28T00:55:35,NaN,24474.0,24935.3,23044.2,NaN,3193.6,3216.4,2724.8,NaN,18430.4,18866.0,17631.3,NaN,2850.0,2852.9,2688.1,NaN
284,2025-12-28 20:00:00-06:00,2025-12-28 21:00:00-06:00,2025-12-28T00:55:35,NaN,24743.0,25303.5,23416.4,NaN,3215.6,3239.7,2749.1,NaN,18640.6,19177.0,17944.9,NaN,2886.8,2886.8,2722.4,NaN
285,2025-12-28 21:00:00-06:00,2025-12-28 22:00:00-06:00,2025-12-28T00:55:35,NaN,22448.6,23010.9,23010.9,NaN,2609.0,2620.4,2620.4,NaN,17086.5,17637.4,17637.4,NaN,2753.1,2753.1,2753.1,NaN
286,2025-12-28 22:00:00-06:00,2025-12-28 23:00:00-06:00,2025-12-28T00:55:35,NaN,21588.1,22148.9,22148.9,NaN,2371.6,2380.2,2380.2,NaN,16463.3,17015.5,17015.5,NaN,2753.2,2753.2,2753.2,NaN


In [21]:
# Solar forecast
df = ercot.get_solar_forecast(start="yesterday")

print(f"Solar Forecast: {len(df):,} records")
df.head()

Solar Forecast: 576 records


,Time,End Time,Posted,Generation System Wide,COP HSL System Wide,STPPF System Wide,PVGRPP System Wide,HSL System Wide
0,2025-12-27 00:00:00-06:00,2025-12-27 01:00:00-06:00,2025-12-27T23:55:36,0.24,0.0,0.0,0.0,0.67
1,2025-12-27 01:00:00-06:00,2025-12-27 02:00:00-06:00,2025-12-27T23:55:36,0.29,0.0,0.0,0.0,0.81
2,2025-12-27 02:00:00-06:00,2025-12-27 03:00:00-06:00,2025-12-27T23:55:36,0.24,0.0,0.0,0.0,0.69
3,2025-12-27 03:00:00-06:00,2025-12-27 04:00:00-06:00,2025-12-27T23:55:36,0.26,0.0,0.0,0.0,0.67
4,2025-12-27 04:00:00-06:00,2025-12-27 05:00:00-06:00,2025-12-27T23:55:36,0.29,0.0,0.0,0.0,0.79


## Date Range Queries

Fetch data across multiple days.

In [4]:
# Get a week of data
df = ercot.get_spp(
    start="2024-12-20",
    end="2024-12-27",
    market=Market.DAY_AHEAD_HOURLY,
    location_type=LocationType.LOAD_ZONE,
)

print(f"Week of DAM SPP: {len(df):,} records")
df.head()

Week of DAM SPP: 1,152 records


,Time,End Time,Location,Price,Market
0,2024-12-26 00:00:00-06:00,2024-12-26 01:00:00-06:00,LZ_AEN,16.64,DAY_AHEAD_HOURLY
1,2024-12-26 00:00:00-06:00,2024-12-26 01:00:00-06:00,LZ_CPS,16.76,DAY_AHEAD_HOURLY
2,2024-12-26 00:00:00-06:00,2024-12-26 01:00:00-06:00,LZ_HOUSTON,16.72,DAY_AHEAD_HOURLY
3,2024-12-26 00:00:00-06:00,2024-12-26 01:00:00-06:00,LZ_LCRA,16.78,DAY_AHEAD_HOURLY
4,2024-12-26 00:00:00-06:00,2024-12-26 01:00:00-06:00,LZ_NORTH,16.75,DAY_AHEAD_HOURLY


## API Reference

| Method | Description | Markets |
|--------|-------------|--------|
| `get_spp()` | Settlement Point Prices | REAL_TIME_15_MIN, DAY_AHEAD_HOURLY |
| `get_lmp()` | Locational Marginal Prices | REAL_TIME_SCED, DAY_AHEAD_HOURLY |
| `get_as_prices()` | Ancillary Service MCPC | - |
| `get_as_plan()` | AS Requirements | - |
| `get_shadow_prices()` | Transmission Constraints | REAL_TIME_SCED, DAY_AHEAD_HOURLY |
| `get_load()` | System Load | - |
| `get_wind_forecast()` | Wind Generation Forecast | - |
| `get_solar_forecast()` | Solar Generation Forecast | - |